### Notebook to build a specials measure

A notebook to help build a specials measure.

- [Imports](#IMPORTS)
- [Availability restriction column analysis](#f1)
- [Explore availability restriction column data](#f2)
- [Restricted Availability - 2](#f2-2)

<a id='IMPORTS'></a>
### Imports
Import libraries required for analysis

In [1]:
#import libraries required for analysis
from ebmdatalab import bq
import os
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

#set to display all rows in data
#pd.set_option('display.max_rows', None)
pd.option_context('display.float_format', '{:,.2f}'.format)

# Set display options for full width
pd.set_option('display.max_colwidth', None)  # Show full column content
pd.set_option('display.width', None)        # Adjust display width to the notebook's width

<a id='f1'></a>
### Availability restriction column analysis
AMP table contains a column 'avail_restrict'. This uses a code to detail availability of products.
The table 'availabilityrestriction' details the meaning behind these codes.

In [2]:
sql = '''
SELECT * FROM dmd.availabilityrestriction
'''
avail_rest_DF = bq.cached_read(sql, csv_path=os.path.join('../','data','avail_rest.csv'))
avail_rest_DF = avail_rest_DF.fillna('None')
avail_rest_DF

,cd,descr
0,1,None
1,2,Restricted Availability
2,3,Individual Patient Supply
3,4,Imported
4,5,Clinical Trial
5,6,Special
6,7,Extemp
7,9,Not available
8,10,For Patient Safety Consideration


We can count the number of AMPs associated with each avail_restrict value.

In [3]:
sql = '''
WITH amp_ar_count AS (
    SELECT avail_restrict, COUNT(*) AS row_count
    FROM `ebmdatalab.dmd.amp` AS amp
    GROUP BY avail_restrict
)
SELECT
    ar.cd,
    ar.descr, 
    SUM(COALESCE(amp_ar_count.row_count, 0)) AS row_count
FROM `dmd.availabilityrestriction` AS ar
LEFT JOIN amp_ar_count
ON ar.cd = amp_ar_count.avail_restrict
GROUP BY ar.descr, ar.cd
ORDER BY ar.cd;
'''
avail_restrict_df = bq.cached_read(sql, csv_path=os.path.join('../','data','avail_restrict_df.csv'))
avail_restrict_df

,cd,descr,row_count
0,1,NaN,89895
1,2,Restricted Availability,56
2,3,Individual Patient Supply,24
3,4,Imported,659
4,5,Clinical Trial,1
5,6,Special,7281
6,7,Extemp,0
7,9,Not available,61299
8,10,For Patient Safety Consideration,4


We can exclude items where avail_restrict = 1 (no restriction) and 9 (not available). 
There are also zero items with avail_restrict = 7 (Extemp).
The remaining values are worth exploring in more detail to better understand.

<a id='f2'></a>
### Explore availability restriction column data
We can extract the rows with avail_restrict values not 1, 7, 9 to explore further what products they contain. We can then try to identify the significance of these products by looking for number of items and spend in the English Prescribing data.

<a id='f2-2'></a>
#### Restricted Availability (avail_restrict 2)

In [4]:
sql = '''
SELECT id, nm, descr, bnf_code 
FROM `ebmdatalab.dmd.amp` 
WHERE avail_restrict = 2
'''
avail_rest_2_DF = bq.cached_read(sql, csv_path=os.path.join('../','data','avail_rest_2.csv'))
avail_rest_2_DF

,id,nm,descr,bnf_code
0,41729811000001105,Vyvgart 400mg/20ml concentrate for solution for infusion vials,Vyvgart 400mg/20ml concentrate for solution for infusion vials (Argenx UK Ltd),NaN
1,42388711000001103,Nexpovio 20mg tablets,Nexpovio 20mg tablets (Menarini Stemline UK Ltd),NaN
2,42527511000001100,Livmarli 9.5mg/ml oral solution,Livmarli 9.5mg/ml oral solution (Mirum Pharmaceuticals International BV),NaN
3,42702411000001107,Vyvgart 1000mg/5.6ml solution for injection vials,Vyvgart 1000mg/5.6ml solution for injection vials (Argenx UK Ltd),NaN
4,43792511000001100,Leqembi 200mg/2ml concentrate for solution for infusion vials,Leqembi 200mg/2ml concentrate for solution for infusion vials (Eisai Ltd),NaN
5,43792811000001102,Leqembi 500mg/5ml concentrate for solution for infusion vials,Leqembi 500mg/5ml concentrate for solution for infusion vials (Eisai Ltd),NaN
6,44145611000001106,Comirnaty KP.2 adults and adolescents from 12 years COVID-19 mRNA Vaccine 30micrograms/0.3ml dose dispersion for injection multidose vials,Comirnaty KP.2 adults and adolescents from 12 years COVID-19 mRNA Vaccine 30micrograms/0.3ml dose dispersion for injection multidose vials (Pfizer Ltd),NaN
7,42977611000001107,Clozapine 25mg tablets,Clozapine 25mg tablets (Viatris UK Healthcare Ltd),0402010C0AAAAAA
8,42977211000001105,Clozapine 100mg tablets,Clozapine 100mg tablets (Viatris UK Healthcare Ltd),0402010C0AAABAB
9,3830311000001101,Clozaril 25mg tablets,Clozaril 25mg tablets (Viatris UK Healthcare Ltd),0402010C0BBAAAA


In [5]:
sql = '''
SELECT bnf_code, 
       bnf_name, 
       SUM(items) AS items, 
       SUM(actual_cost) AS cost
FROM `ebmdatalab.hscic.normalised_prescribing`
WHERE bnf_code IN (
    SELECT DISTINCT bnf_code 
    FROM `ebmdatalab.dmd.amp` 
    WHERE avail_restrict = 2
)
GROUP BY bnf_code, bnf_name
ORDER BY items DESC
'''
avail_rest_2_usage_DF = bq.cached_read(sql, csv_path=os.path.join('../','data','avail_restrict_2_usage.csv'))

avail_rest_2_usage_DF

,bnf_code,bnf_name,items,cost
0,0402010C0AAABAB,Clozapine 100mg tablets,12588,504203.85949
1,090607000BBLJA0,Healthy Start Children's Vitamin drops,9901,28589.76976
2,0402010C0AAAAAA,Clozapine 25mg tablets,7506,58419.94802
3,0402010C0AAABAB,Clozapine_Tab 100mg,5808,326376.58000
4,0402010C0AAAAAA,Clozapine_Tab 25mg,3425,32652.56000
5,0402010C0BCABAB,Denzapine 100mg tablets,2833,144920.28822
6,090607000BBLPA0,Healthy Start Women's Vitamin tablets,2633,3863.20610
7,0402010C0BBABAB,Clozaril 100mg tablets,1967,60604.36986
8,0402010C0BBABAB,Clozaril_Tab 100mg,1840,90906.12000
9,1404000AMBBAAAA,Gardasil 9 vacc inj 0.5ml pre-filled syringes,1451,172242.16817


**Restricted availability** denotes medications which are only prescribed from specialist services - not relevent to "specials" measure

<a id='f2-3'></a>
#### Individual Patient Supply (avail_restrict 3)

In [6]:
sql = '''
SELECT id, nm, descr, bnf_code 
FROM `ebmdatalab.dmd.amp` 
WHERE avail_restrict = 3
'''
avail_rest_3_DF = bq.cached_read(sql, csv_path=os.path.join('../','data','avail_rest_3.csv'))
avail_rest_3_DF

,id,nm,descr,bnf_code
0,15514711000001108,"Factor XI 1,000unit powder and solvent for solution for infusion vials","Factor XI 1,000unit powder and solvent for solution for infusion vials (Bio Products Laboratory Ltd)",NaN
1,35820711000001107,Crysvita 10mg/1ml solution for injection vials,Crysvita 10mg/1ml solution for injection vials (Kyowa Kirin Ltd),NaN
2,35821111000001100,Crysvita 20mg/1ml solution for injection vials,Crysvita 20mg/1ml solution for injection vials (Kyowa Kirin Ltd),NaN
3,35821511000001109,Crysvita 30mg/1ml solution for injection vials,Crysvita 30mg/1ml solution for injection vials (Kyowa Kirin Ltd),NaN
4,39782811000001104,Vistogard 10g oral granules sachets,Vistogard 10g oral granules sachets (Protherics Medicines Development Ltd),NaN
5,42561311000001109,Oxervate 20micrograms/ml eye drops 1ml unit dose,Oxervate 20micrograms/ml eye drops 1ml unit dose (Chapper Healthcare),NaN
6,43998011000001102,Yorvipath 168micrograms/0.56ml solution for injection pre-filled pens,Yorvipath 168micrograms/0.56ml solution for injection pre-filled pens (Ascendis Pharma UK Ltd),NaN
7,43998811000001108,Yorvipath 294micrograms/0.98ml solution for injection pre-filled pens,Yorvipath 294micrograms/0.98ml solution for injection pre-filled pens (Ascendis Pharma UK Ltd),NaN
8,43999211000001102,Yorvipath 420micrograms/1.4ml solution for injection pre-filled pens,Yorvipath 420micrograms/1.4ml solution for injection pre-filled pens (Ascendis Pharma UK Ltd),NaN
9,44013311000001103,Dasiglucagon 4mg/1ml solution for infusion vials,Dasiglucagon 4mg/1ml solution for infusion vials (WEP Clinical),NaN


In [7]:
sql = '''
SELECT bnf_code, 
       bnf_name, 
       SUM(items) AS items, 
       SUM(actual_cost) AS cost
FROM `ebmdatalab.hscic.normalised_prescribing`
WHERE bnf_code IN (
    SELECT DISTINCT bnf_code 
    FROM `ebmdatalab.dmd.amp` 
    WHERE avail_restrict = 3
)
GROUP BY bnf_code, bnf_name
ORDER BY items DESC
'''
avail_rest_3_usage_DF = bq.cached_read(sql, csv_path=os.path.join('../','data','avail_restrict_3_usage.csv'))

avail_rest_3_usage_DF

,bnf_code,bnf_name,items,cost
0,0401030E0BBAAAA,Sodium Amytal 60mg capsules,7481,673365.11672
1,0401030T0BCAAAC,Tuinal 100mg Pulvules,7313,696800.10164
2,0401030T0BCAAAC,Tuinal_Pulvule 100mg,7279,507441.91000
3,0401030E0BBAAAA,Sod Amytal_Pulvule 60mg,6063,848094.45000
4,0401030H0BBAAAB,Soneryl_Tab 100mg,2650,233409.60000
5,0401030H0BBAAAB,Soneryl 100mg tablets,1859,211325.73693
6,1311070S0BDAEAA,BioBag 50 2.5cm x 4cm dressing kit,1619,372638.81023
7,0401030T0BBABAB,Seconal Sodium 100mg capsules,1610,332084.50744
8,0401030T0BBABAB,Seconal Sod_Pulvule 100mg,1519,241424.19000
9,1311070S0BDADAA,BioBag 100 4cm x 5cm dressing kit,1395,353998.12394


**Individual Patient Supply** denotes medications which are only prescribed for a select number of patients

<a id='f2-4'></a>
#### Imported (avail_restrict 4)

In [8]:
sql = '''
SELECT id, nm, descr, bnf_code 
FROM `ebmdatalab.dmd.amp` test_DF
WHERE avail_restrict = 4
'''
avail_rest_4_DF = bq.cached_read(sql, csv_path=os.path.join('../','data','avail_rest_4.csv'))
avail_rest_4_DF

,id,nm,descr,bnf_code
0,29175011000001104,Iperten 10mg tablets,Iperten 10mg tablets (Imported (France)),NaN
1,29193611000001104,Proctolog ointment,Proctolog ointment (Imported (Spain)),NaN
2,31584411000001103,Dronabinol 2.5mg capsules,Dronabinol 2.5mg capsules (Imported (United States)),NaN
3,31736011000001109,Lumitene 30mg capsules,Lumitene 30mg capsules (Imported (United States)),NaN
4,32641911000001105,Triapten Antiviral 2% cream,Triapten Antiviral 2% cream (Imported (Germany)),NaN
...,...,...,...,...
654,19390111000001109,Abnoba Viscum Q 200micrograms/1ml solution for injection ampoules,Abnoba Viscum Q 200micrograms/1ml solution for injection ampoules (Imported (Germany)),190203000BBAAA0
655,19390511000001100,Abnoba Viscum Q 20mg/1ml solution for injection ampoules,Abnoba Viscum Q 20mg/1ml solution for injection ampoules (Imported (Germany)),190203000BBAAA0
656,19390811000001102,Abnoba Viscum Q 20micrograms/1ml solution for injection ampoules,Abnoba Viscum Q 20micrograms/1ml solution for injection ampoules (Imported (Germany)),190203000BBAAA0
657,19391111000001103,Abnoba Viscum Q 2mg/1ml solution for injection ampoules,Abnoba Viscum Q 2mg/1ml solution for injection ampoules (Imported (Germany)),190203000BBAAA0


In [9]:
sql = '''
SELECT bnf_code, 
       bnf_name, 
       SUM(items) AS items, 
       SUM(actual_cost) AS cost
FROM `ebmdatalab.hscic.normalised_prescribing`
WHERE bnf_code IN (
    SELECT DISTINCT bnf_code 
    FROM `ebmdatalab.dmd.amp` 
    WHERE avail_restrict = 4
)
GROUP BY bnf_code, bnf_name
ORDER BY items DESC
'''
avail_rest_4_usage_DF = bq.cached_read(sql, csv_path=os.path.join('../','data','avail_restrict_4_usage.csv'))

with pd.option_context('display.max_rows', 100):
    with pd.option_context('display.float_format', '{:,.2f}'.format):
        display(avail_rest_4_usage_DF.head(100))

Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████|


,bnf_code,bnf_name,items,cost
0,0602010V0AABZBZ,Levothyroxine sodium 100microgram tablets,121398229,"194,216,666.86"
1,0602010V0AABXBX,Levothyroxine sodium 50microgram tablets,109805716,"169,907,058.25"
2,0602010V0AABWBW,Levothyroxine sodium 25microgram tablets,107289528,"245,342,049.35"
3,0602010V0AABZBZ,Levothyrox Sod_Tab 100mcg,31541510,"58,139,478.48"
4,0602010V0AABXBX,Levothyrox Sod_Tab 50mcg,28604796,"52,320,495.91"
5,0602010V0AABWBW,Levothyrox Sod_Tab 25mcg,27607357,"91,205,218.91"
6,1003020AAAAABAB,Capsaicin 0.025% cream,1600933,"30,953,424.15"
7,1003020AAAAAAAA,Capsaicin 0.075% cream,799709,"12,781,324.05"
8,0301020I0AAACAC,Ipratropium bromide 500micrograms/2ml neb liq ud vials,737865,"9,505,273.87"
9,1003020AAAAABAB,Capsaicin_Crm 0.025%,476869,"9,238,076.82"


**Imported** denotes imported medications. The 'descr' field details Imported and the country imported from. However the BNF code isn't unique to imported products - for example 0602010V0AABZBZ	Levothyroxine sodium 100microgram tablets - seems to include standard UK product and imported products.

<a id='f2-5'></a>
#### Clinical Trial (avail_restrict 5)

In [10]:
sql = '''
SELECT id, nm, descr, bnf_code 
FROM `ebmdatalab.dmd.amp` 
WHERE avail_restrict = 5
'''
avail_rest_5_DF = bq.cached_read(sql, csv_path=os.path.join('../','data','avail_rest_5.csv'))
avail_rest_5_DF

,id,nm,descr,bnf_code
0,41823211000001105,COVID-19 Vaccine Moderna (mRNA-1283.222) 10micrograms/0.2ml dose solution for injection vials,"COVID-19 Vaccine Moderna (mRNA-1283.222) 10micrograms/0.2ml dose solution for injection vials (Moderna, Inc)",1404000ATBRADBJ


In [11]:
sql = '''
SELECT bnf_code, 
       bnf_name, 
       SUM(items) AS items, 
       SUM(actual_cost) AS cost
FROM `ebmdatalab.hscic.normalised_prescribing`
WHERE bnf_code IN (
    SELECT DISTINCT bnf_code 
    FROM `ebmdatalab.dmd.amp` 
    WHERE avail_restrict = 5
)
GROUP BY bnf_code, bnf_name
ORDER BY items DESC
'''
avail_rest_5_usage_DF = bq.cached_read(sql, csv_path=os.path.join('../','data','avail_restrict_5_usage.csv'))

avail_rest_5_usage_DF

,bnf_code,bnf_name,items,cost


**Clinical Trial** denotes a medication used for clinical trial. Only one product, no prescribing.

<a id='f2-6'></a>
#### Specials (avail_restrict 6)

In [12]:
sql = '''
SELECT id, nm, descr, bnf_code 
FROM `ebmdatalab.dmd.amp` 
WHERE avail_restrict = 6
'''
avail_rest_6_DF = bq.cached_read(sql, csv_path=os.path.join('../','data','avail_rest_6.csv'))
avail_rest_6_DF

,id,nm,descr,bnf_code
0,19980211000001105,Potassium ascorbate 10% eye drops preservative free,Potassium ascorbate 10% eye drops preservative free (Special Order),NaN
1,20399911000001104,Coal tar 30% in Emulsifying ointment,Coal tar 30% in Emulsifying ointment (Special Order),NaN
2,25192611000001104,Iodine 1% in Boric acid ear powder,Iodine 1% in Boric acid ear powder (Special Order),NaN
3,25407511000001104,Hydrochloric acid 18% dental solution,Hydrochloric acid 18% dental solution (Special Order),NaN
4,25566311000001101,Fentanyl 24.75mg/990ml infusion bags,Fentanyl 24.75mg/990ml infusion bags (Special Order),NaN
...,...,...,...,...
7276,32284311000001104,Cade oil 12.5% in Emulsifying ointment,Cade oil 12.5% in Emulsifying ointment (Special Order),190605000AACECE
7277,24551811000001101,Ginger syrup,Ginger syrup (Special Order),190607000AAAGAG
7278,18159811000001109,Eosin 1% solution,Eosin 1% solution (Special Order),190800000AAAPAP
7279,17656311000001103,Eosin 2% solution,Eosin 2% solution (Drug Tariff Special Order),190800000AABTBT


In [13]:
sql = '''
SELECT bnf_code, 
       bnf_name, 
       SUM(items) AS items, 
       SUM(actual_cost) AS cost
FROM `ebmdatalab.hscic.normalised_prescribing`
WHERE bnf_code IN (
    SELECT DISTINCT bnf_code 
    FROM `ebmdatalab.dmd.amp` 
    WHERE avail_restrict = 6
)
GROUP BY bnf_code, bnf_name
ORDER BY items DESC
'''
avail_rest_6_usage_DF = bq.cached_read(sql, csv_path=os.path.join('../','data','avail_restrict_6_usage.csv'))

with pd.option_context('display.max_rows', 100):
    with pd.option_context('display.float_format', '{:,.2f}'.format):
        display(avail_rest_6_usage_DF.head(100))

,bnf_code,bnf_name,items,cost
0,0410030C0AAAFAF,Methadone 1mg/ml oral solution sugar free,9126099,"61,590,579.30"
1,0410030C0AAAAAA,Methadone 1mg/ml oral solution,4742757,"31,371,963.65"
2,0410030C0AAAFAF,Methadone HCl_Mix 1mg/1ml S/F,2017025,"15,008,152.64"
3,0410030C0AAAAAA,Methadone HCl_Mix 1mg/1ml,1869833,"15,461,493.30"
4,0901020D0AAAHAH,Cyanocobalamin 1mg modified-release tablets,1216501,"7,558,454.02"
5,0906040G0AAELEL,Colecalciferol 400unit tablets,693619,"5,557,402.90"
6,0407010Q0AAAAAA,Co-Proxamol_Tab 32.5mg/325mg,551243,"15,051,015.47"
7,0401010ADAACZCZ,Melatonin 1mg/ml oral solution sugar free,484045,"70,526,833.96"
8,0407010Q0AAAAAA,Co-proxamol 32.5mg/325mg tablets,350255,"35,862,526.11"
9,0906040G0AACRCR,"Colecalciferol 20,000unit tablets",297544,"5,421,599.07"


<a id='f2-10'></a>
#### For Patient Safety Consideration (avail_restrict 10)

In [14]:
sql = '''
SELECT id, nm, descr, bnf_code 
FROM `ebmdatalab.dmd.amp` 
WHERE avail_restrict = 10
'''
avail_rest_10_DF = bq.cached_read(sql, csv_path=os.path.join('../','data','avail_rest_10.csv'))
avail_rest_10_DF

,id,nm,descr,bnf_code
0,26345611000001105,Phenol 90% cutaneous solution - should NOT be prescribed,Phenol 90% cutaneous solution - should NOT be prescribed (Special Order),NaN
1,26346211000001102,Phenol 95% cutaneous solution - should NOT be prescribed,Phenol 95% cutaneous solution - should NOT be prescribed (Special Order),NaN
2,29718111000001107,Phenol 80% cutaneous solution - should NOT be prescribed,Phenol 80% cutaneous solution - should NOT be prescribed (Special Order),NaN
3,22235811000001100,Phenol liquefied - should NOT be prescribed,Phenol liquefied - should NOT be prescribed (Special Order),1311050P0AAABAB


In [15]:
sql = '''
SELECT bnf_code, 
       bnf_name, 
       SUM(items) AS items, 
       SUM(actual_cost) AS cost
FROM `ebmdatalab.hscic.normalised_prescribing`
WHERE bnf_code IN (
    SELECT DISTINCT bnf_code 
    FROM `ebmdatalab.dmd.amp` 
    WHERE avail_restrict = 10
)
GROUP BY bnf_code, bnf_name
ORDER BY items DESC
'''
avail_rest_10_usage_DF = bq.cached_read(sql, csv_path=os.path.join('../','data','avail_restrict_4_usage.csv'))

avail_rest_10_usage_DF

Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████|


,bnf_code,bnf_name,items,cost
0,1311050P0AAABAB,Phenol liquefied,78,4465.70855
1,1311050P0AAABAB,Phenol Liquefied_,69,2016.57000


**For Patient Safety Consideration** denotes medication which appear not to be recommended for prescribing (all denote "should NOT be prescribed").

### Summary of avail_restrict  


| cd  | descr                             | row_count | comments                       |
|-----|-----------------------------------|-----------|--------------------------------|
| 1   | None                              | 89895     | No restrictions on availability     |
| 2   | Restricted Availability           | 56        | medications which are only prescribed from specialist services e.g. clozapine           |
| 3   | Individual Patient Supply         | 24        | Only prescribed for a select number of patients e.g. sodium amytal, BioBag   |
| 4   | Imported                          | 659       | Sourced from another country   |
| 5   | Clinical Trial                    | 1         | For research purposes          |
| 6   | Special                           | 7281      | Specials products     |
| 7   | Extemp                            | 0         | No items currently listed here|
| 9   | Not available                     | 61299     | Currently unavailable products          |
| 10  | For Patient Safety Consideration  | 4         | Products limited for safety reasons     |


<a id='#f3'></a>
### Focus on specials
Firstly look at how many specials are denoted by BNF code that only signifies a special.


In [16]:
sql = '''
SELECT 
    bnf_code,
    SUM(CASE WHEN avail_restrict = 6 THEN 1 ELSE 0 END) AS special_count,
    SUM(CASE WHEN avail_restrict != 6 THEN 1 ELSE 0 END) AS non_special_count
FROM 
    `ebmdatalab.dmd.amp`
WHERE
    bnf_code IS NOT NULL
GROUP BY 
    bnf_code
HAVING 
    special_count > 0
ORDER BY 
    special_count DESC, non_special_count DESC;
'''
spec_non_spec_count_DF = bq.cached_read(sql, csv_path=os.path.join('../','data','spec_non_spec_count.csv'))
spec_non_spec_count_DF

,bnf_code,special_count,non_special_count
0,190203000BBAAA0,46,397
1,1305020C0AAFPFP,6,2
2,1307000M0AABVBV,4,1
3,1305020C0AAAVAV,4,0
4,1305020C0AABSBS,4,0
...,...,...,...
4432,190601000AABEBE,1,0
4433,190605000AACDCD,1,0
4434,190605000AACECE,1,0
4435,190800000AAAPAP,1,0


In [17]:
sql = '''
SELECT 
    amp.bnf_code,
    SUM(CASE WHEN amp.avail_restrict = 6 THEN 1 ELSE 0 END) AS special_count,
    SUM(CASE WHEN (amp.avail_restrict != 6 AND amp.avail_restrict != 9) THEN 1 ELSE 0 END) AS non_special_count,
    STRING_AGG(DISTINCT CASE WHEN amp.avail_restrict = 6 THEN amp.descr ELSE NULL END, ', ') AS special_names,
    STRING_AGG(DISTINCT CASE WHEN (amp.avail_restrict != 6 AND amp.avail_restrict != 9) THEN amp.descr ELSE NULL END, ', ') AS non_special_names,
    STRING_AGG(DISTINCT vmp.nm, ', ') AS vpm_names
FROM 
    `ebmdatalab.dmd.amp` AS amp
LEFT JOIN 
    `ebmdatalab.dmd.vmp` AS vmp
ON 
    amp.bnf_code = vmp.bnf_code
WHERE
    amp.bnf_code IS NOT NULL
GROUP BY 
    amp.bnf_code
HAVING 
    special_count > 0
ORDER BY 
    non_special_count DESC;
'''
spec_non_spec_count_named_DF = bq.cached_read(sql, csv_path=os.path.join('../','data','spec_non_spec_count_named.csv'))

with pd.option_context('display.max_rows', 100):
    display(spec_non_spec_count_named_DF.head(100))


bnf_code  special_count  non_special_count  \
0   190203000BBAAA0          16238              37771   
1   0401010ADAACZCZ              1                  7   
2   0410030C0AAAAAA              1                  5   
3   0410030C0AAAFAF              1                  5   
4   0407010H0AAA5A5              1                  4   
5   0906012B0AAACAC              1                  2   
6   0906025P0AABFBF              1                  2   
7   0402010AFAAAAAA              1                  2   
8   0702020T0AAAFAF              1                  1   
9   090501300AABIBI              1                  1   
10  0905013F0AAABAB              1                  1   
11  090504100AABBBB              1                  1   
12  0906022K0AAAPAP              1                  1   
13  0913445A0AAAGAG              1                  1   
14  0203020U0AAAGAG              1                  1   
15  0401010ADAAACAC              1                  1   
16  0401010ADAAAHAH              1                  1   
17  0401010ADAAAJAJ              1                  1   
18  0401010ADAAAKAK              1                  1   
19  0401010ADAAAQAQ              1                  1   
20  0401010ADAABIBI              1                  1   
21  0401010ADAABQBQ              1                  1   
22  0402010AHAAAAAA              1                  1   
23  0406000W0AAAMAM              1                  1   
24  0407020Y0AAADAD              1                  1   
25  0408020W0AAAHAH              1                  1   
26  0409020G0AAACAC              1                  1   
27  0501021E0AAADAD              1                  1   
28  0501021E0AAAEAE              1                  1   
29  0501030F0AAAIAI              1                  1   
30  0501080T0AAAAAA              1                  1   
31  0602010V0AAFWFW              1                  1   
32  0605020E0AAASAS              1                  1   
33  0801030L0AAABAB              1                  1   
34  0801050BUAAAAAA              1                  1   
35  0801050BUAAABAB              1                  1   
36  0803020L0AAAAAA              1                  1   
37  0803020L0AAAGAG              1                  1   
38  0906031C0AAAPAP              1                  1   
39  0906040G0AACHCH              1                  1   
40  0906040G0AAEAEA              1                  1   
41  1001040K0AAADAD              1                  1   
42  1001040P0AAACAC              1                  1   
43  1502010G0AAAUAU              1                  1   
44  0101010C0AAAHAH              1                  0   
45  0101010C0AAAMAM              1                  0   
46  0101010F0AAAUAU              1                  0   
47  0101010I0AAABAB              1                  0   
48  0101010I0AAACAC              1                  0   
49  0101010I0AAADAD              1                  0   
50  0101010I0AAAHAH              1                  0   
51  0101010I0AAAJAJ              1                  0   
52  0101010I0AAALAL              1                  0   
53  0101010I0AAAMAM              1                  0   
54  0101010I0AAATAT              1                  0   
55  0101010I0AAAUAU              1                  0   
56  0101010I0AAAVAV              1                  0   
57  0101010I0AAAXAX              1                  0   
58  0101010I0AAAYAY              1                  0   
59  0101010I0AABGBG              1                  0   
60  0101010I0AABIBI              1                  0   
61  0101010I0AABPBP              1                  0   
62  0101010I0AABQBQ              1                  0   
63  0101010I0AABRBR              1                  0   
64  0101010I0AABSBS              1                  0   
65  0101010I0AABTBT              1                  0   
66  0101010N0AAABAB              1                  0   
67  0101012B0AAALAL              1                  0   
68  0101012B0AAATAT              1                  0   
69  0101012B0AAAUAU  

BNF code 190203000BBAAA0 covers all homeopathic preperations. This is a mix of specials and none specials and cannot be differentiated. Prescribing of these products is minimal - £2,625 across England Nov '23—Oct '24. So will exclude this code (and detail this in any WIM).

Looking further at the list a number of products have 1 special product and 1 imported product listed under the same BNF code - without any none special/none imported product. Given that the imported products would be unlicensed in nature it seems reasonable to include prescribing under these BNF codes. But worth investigating further how imported products link to specials and none specials.

In [18]:
sql = '''
SELECT 
    amp.bnf_code,
    SUM(CASE WHEN (amp.avail_restrict = 6) THEN 1 ELSE 0 END) AS special_count,
    SUM(CASE WHEN (amp.avail_restrict = 4) THEN 1 ELSE 0 END) AS import_count,
    SUM(CASE WHEN (amp.avail_restrict != 6 AND amp.avail_restrict != 9 AND amp.avail_restrict != 4) THEN 1 ELSE 0 END) AS non_special_non_import_count,
    STRING_AGG(DISTINCT CASE WHEN amp.avail_restrict = 6 THEN amp.descr ELSE NULL END, ', ') AS special_names,
    STRING_AGG(DISTINCT CASE WHEN (amp.avail_restrict = 4) THEN amp.descr ELSE NULL END, ', ') AS import_names,
    STRING_AGG(DISTINCT CASE WHEN (amp.avail_restrict != 6 AND amp.avail_restrict != 9 AND amp.avail_restrict != 4) THEN amp.descr ELSE NULL END, ', ') AS non_special_names,
    STRING_AGG(DISTINCT vmp.nm, ', ') AS vpm_names
FROM 
    `ebmdatalab.dmd.amp` AS amp
LEFT JOIN 
    `ebmdatalab.dmd.vmp` AS vmp
ON 
    amp.bnf_code = vmp.bnf_code
WHERE
    amp.bnf_code IS NOT NULL
    AND
    amp.bnf_code != '190203000BBAAA0' --  Exclude homeopathic products as total mix of specials and none specials but minimal impact on cost ~£2600 across England over 12 months
GROUP BY 
    amp.bnf_code
HAVING 
    special_count > 0
    AND
    import_count >0
    AND
    non_special_non_import_count = 0
ORDER BY 
    import_count DESC;
'''
spec_imported_non_spec_count_named_DF = bq.cached_read(sql, csv_path=os.path.join('../','data','spec_imported_non_spec_count_named.csv'))

with pd.option_context('display.max_rows', 100):
    display(spec_imported_non_spec_count_named_DF.head(100))

,bnf_code,special_count,import_count,non_special_non_import_count,special_names,import_names,non_special_names,vpm_names
0,0402010AFAAAAAA,1,2,0,Melperone hydrochloride 100mg tablets (Special Order),"Melperone hydrochloride 100mg tablets (Imported (Sweden)), Melperone hydrochloride 100mg tablets (Imported (Germany))",NaN,Melperone hydrochloride 100mg tablets
1,0203020U0AAAGAG,1,1,0,Quinidine sulfate 200mg tablets (Special Order),Quinidine sulfate 200mg tablets (Imported (United States)),NaN,Quinidine sulfate 200mg tablets
2,0401010ADAAACAC,1,1,0,Melatonin 500microgram capsules (Special Order),Melatonin 500microgram capsules (Imported (United States)),NaN,Melatonin 500microgram capsules
3,0401010ADAAAHAH,1,1,0,Melatonin 2.5mg capsules (Part VIIID Drug Tariff Special Order),Melatonin 2.5mg capsules (Imported (United States)),NaN,Melatonin 2.5mg capsules
4,0401010ADAAAJAJ,1,1,0,Melatonin 3mg modified-release capsules (Part VIIID Drug Tariff Special Order),Melatonin 3mg modified-release capsules (Imported (United States)),NaN,Melatonin 3mg modified-release capsules
5,0401010ADAAAKAK,1,1,0,Melatonin 10mg capsules (Special Order),Melatonin 10mg capsules (Imported (United States)),NaN,Melatonin 10mg capsules
6,0401010ADAAAQAQ,1,1,0,Melatonin 3mg modified-release tablets (Special Order),Melatonin 3mg modified-release tablets (Imported (United States)),NaN,Melatonin 3mg modified-release tablets
7,0401010ADAABIBI,1,1,0,Melatonin 3mg lozenges sugar free (Special Order),Melatonin 3mg lozenges sugar free (Imported (United States)),NaN,Melatonin 3mg lozenges sugar free
8,0401010ADAABQBQ,1,1,0,Melatonin 1mg capsules (Special Order),Melatonin 1mg capsules (Imported (United States)),NaN,Melatonin 1mg capsules
9,0402010AHAAAAAA,1,1,0,Clotiapine 40mg tablets (Special Order),Clotiapine 40mg tablets (Imported (South Africa)),NaN,Clotiapine 40mg tablets


All BNF codes with only AMPs which have a special or imported status would be reasonable to include in the measure.

We should also look at identifying which BNF codes have an imported product and none special product.

In [19]:
sql = '''
SELECT 
    amp.bnf_code,
    SUM(CASE WHEN (amp.avail_restrict = 6) THEN 1 ELSE 0 END) AS special_count,
    SUM(CASE WHEN (amp.avail_restrict = 4) THEN 1 ELSE 0 END) AS import_count,
    SUM(CASE WHEN (amp.avail_restrict != 6 AND amp.avail_restrict != 9 AND amp.avail_restrict != 4) THEN 1 ELSE 0 END) AS non_special_non_import_count,
    STRING_AGG(DISTINCT CASE WHEN amp.avail_restrict = 6 THEN amp.descr ELSE NULL END, ', ') AS special_names,
    STRING_AGG(DISTINCT CASE WHEN (amp.avail_restrict = 4) THEN amp.descr ELSE NULL END, ', ') AS import_names,
    STRING_AGG(DISTINCT CASE WHEN (amp.avail_restrict != 6 AND amp.avail_restrict != 9 AND amp.avail_restrict != 4) THEN amp.descr ELSE NULL END, ', ') AS non_special_names,
    STRING_AGG(DISTINCT vmp.nm, ', ') AS vpm_names
FROM 
    `ebmdatalab.dmd.amp` AS amp
LEFT JOIN 
    `ebmdatalab.dmd.vmp` AS vmp
ON 
    amp.bnf_code = vmp.bnf_code
WHERE
    amp.bnf_code IS NOT NULL
    AND
    amp.bnf_code != '190203000BBAAA0' --  Exclude homeopathic products as total mix of specials and none specials but minimal impact on cost ~£2600 across England over 12 months
GROUP BY 
    amp.bnf_code
HAVING 
    import_count >0
    AND
    non_special_non_import_count > 0
ORDER BY 
    import_count DESC;
'''
imported_non_spec_count_named_DF = bq.cached_read(sql, csv_path=os.path.join('../','data','imported_non_spec_count_named.csv'))

with pd.option_context('display.max_rows', 100):
    display(imported_non_spec_count_named_DF.head(100))

,bnf_code,special_count,import_count,non_special_non_import_count,special_names,import_names,non_special_names,vpm_names
0,0401010ADAABRBR,0,2,3,NaN,"Melatonin 3mg capsules (Imported (United States)), Melatonin 3mg capsules (Imported (Denmark))","Melatonin 3mg capsules (Colonis Pharma Ltd), Melatonin 3mg capsules (Alliance Healthcare (Distribution) Ltd), Melatonin 3mg capsules (A A H Pharmaceuticals Ltd)",Melatonin 3mg capsules
1,0102000J0AAAAAA,0,1,6,NaN,Dicycloverine 10mg/5ml oral solution (Imported (United States)),"Dicycloverine 10mg/5ml oral solution (Thame Laboratories Ltd), Dicycloverine 10mg/5ml oral solution (Alliance Healthcare (Distribution) Ltd), Dicycloverine 10mg/5ml oral solution (A A H Pharmaceuticals Ltd), Dicycloverine 10mg/5ml oral solution (Phoenix Healthcare Distribution Ltd), Dicycloverine 10mg/5ml oral solution (Teva UK Ltd), Dicycloverine 10mg/5ml oral solution (Medihealth (Northern) Ltd)",Dicycloverine 10mg/5ml oral solution
2,0203020P0AAANAN,0,1,3,NaN,Mexiletine hydrochloride 200mg (Mexiletine 167mg) capsules (Imported (United States)),"Mexiletine hydrochloride 200mg (Mexiletine 167mg) capsules (Colonis Pharma Ltd), Mexiletine hydrochloride 200mg (Mexiletine 167mg) capsules (A A H Pharmaceuticals Ltd), Mexiletine hydrochloride 200mg (Mexiletine 167mg) capsules (Alliance Healthcare (Distribution) Ltd)",Mexiletine hydrochloride 200mg (Mexiletine 167mg) capsules
3,0301020I0AAACAC,0,1,6,NaN,Ipratropium bromide 500micrograms/2ml nebuliser liquid unit dose vials (Imported),"Ipratropium bromide 500micrograms/2ml nebuliser liquid unit dose vials (A A H Pharmaceuticals Ltd), Ipratropium bromide 500micrograms/2ml nebuliser liquid unit dose vials (Alliance Healthcare (Distribution) Ltd), Ipratropium bromide 500micrograms/2ml nebuliser liquid unit dose vials (Accord-UK Ltd), Ipratropium bromide 500micrograms/2ml nebuliser liquid unit dose vials (Sigma Pharmaceuticals Plc), Ipratropium bromide 500micrograms/2ml nebuliser liquid unit dose vials (Phoenix Healthcare Distribution Ltd), Ipratropium bromide 500micrograms/2ml nebuliser liquid unit dose vials (Medihealth (Northern) Ltd)",Ipratropium bromide 500micrograms/2ml nebuliser liquid unit dose vials
4,0301020I0AAAFAF,0,1,6,NaN,Ipratropium bromide 250micrograms/1ml nebuliser liquid unit dose vials (Imported),"Ipratropium bromide 250micrograms/1ml nebuliser liquid unit dose vials (A A H Pharmaceuticals Ltd), Ipratropium bromide 250micrograms/1ml nebuliser liquid unit dose vials (Alliance Healthcare (Distribution) Ltd), Ipratropium bromide 250micrograms/1ml nebuliser liquid unit dose vials (Accord-UK Ltd), Ipratropium bromide 250micrograms/1ml nebuliser liquid unit dose vials (Sigma Pharmaceuticals Plc), Ipratropium bromide 250micrograms/1ml nebuliser liquid unit dose vials (Phoenix Healthcare Distribution Ltd), Ipratropium bromide 250micrograms/1ml nebuliser liquid unit dose vials (Medihealth (Northern) Ltd)",Ipratropium bromide 250micrograms/1ml nebuliser liquid unit dose vials
5,0401010ADAAAEAE,0,1,4,NaN,Melatonin 2mg capsules (Imported (Denmark)),"Melatonin 2mg capsules (Colonis Pharma Ltd), Melatonin 2mg capsules (Alliance Healthcare (Distribution) Ltd), Melatonin 2mg capsules (A A H Pharmaceuticals Ltd), Melatonin 2mg capsules (Medihealth (Northern) Ltd)",Melatonin 2mg capsules
6,0401010ADAABSBS,0,1,3,NaN,Melatonin 5mg capsules (Imported (United States)),"Melatonin 5mg capsules (Colonis Pharma Ltd), Melatonin 5mg capsules (Alliance Healthcare (Distribution) Ltd), Melatonin 5mg capsules (A A H Pharmaceuticals Ltd)",Melatonin 5mg capsules
7,0502050B0AAABAB,0,1,3,NaN,Griseofulvin 125mg tablets (Imported (United States)),"Griseofulvin 125mg tablets (Essential Generics Ltd), Griseofulvin 125mg tablets (Alliance Healthcare (Distribution) Ltd), Griseofulvin 125mg tablets (A A H Pharmaceuticals Ltd)",Griseofulvin 125mg tablets
8,0502050B0AAACAC,0,1,3,NaN,Griseofulvin 500mg tablets (Imported (United States)),"Griseofulvin 500mg tablets (Essential Generics Ltd), Griseofu

Only a small number of products. Add spend column to understand significance of the spending against that BNF code.

In [20]:
sql = '''
SELECT bnf_code, 
       bnf_name, 
       SUM(items) AS items, 
       SUM(actual_cost) AS cost
FROM `ebmdatalab.hscic.normalised_prescribing`
WHERE bnf_code IN (
    WITH aggregated_data AS (
        SELECT 
            amp.bnf_code,
            SUM(CASE WHEN (amp.avail_restrict = 6) THEN 1 ELSE 0 END) AS special_count,
            SUM(CASE WHEN (amp.avail_restrict = 4) THEN 1 ELSE 0 END) AS import_count,
            SUM(CASE WHEN (amp.avail_restrict != 6 AND amp.avail_restrict != 9 AND amp.avail_restrict != 4) THEN 1 ELSE 0 END) AS non_special_non_import_count
        FROM 
            `ebmdatalab.dmd.amp` AS amp
        WHERE
            amp.bnf_code IS NOT NULL
            AND amp.bnf_code != '190203000BBAAA0'
        GROUP BY 
            amp.bnf_code
    )
    SELECT DISTINCT bnf_code
    FROM aggregated_data
    WHERE import_count > 0
    AND non_special_non_import_count > 0
)
AND month >='2023-11-01'
AND month < '2024-11-01'
GROUP BY bnf_code, bnf_name
ORDER BY cost DESC
'''
mixed_imported_DF = bq.cached_read(sql, csv_path=os.path.join('../','data','mixed_imported.csv'))
with pd.option_context('display.max_rows', 100):
    with pd.option_context('display.float_format', '{:,.2f}'.format):
        display(mixed_imported_DF.head(100))

,bnf_code,bnf_name,items,cost
0,0602010V0AABZBZ,Levothyroxine sodium 100microgram tablets,12020168,"9,503,331.59"
1,0602010V0AABWBW,Levothyroxine sodium 25microgram tablets,9947139,"8,730,341.78"
2,0602010V0AABXBX,Levothyroxine sodium 50microgram tablets,10253041,"7,767,556.64"
3,0401010ADAAAEAE,Melatonin 2mg capsules,33438,"2,683,691.49"
4,0401010ADAABRBR,Melatonin 3mg capsules,17362,"1,444,806.38"
5,0203020P0AAANAN,Mexiletine hydrochloride 200mg (Mexiletine 167mg) capsules,186,"810,406.93"
6,0401010ADAABSBS,Melatonin 5mg capsules,4588,"651,839.03"
7,0102000J0AAAAAA,Dicycloverine 10mg/5ml oral solution,1239,"602,816.85"
8,0301020I0AAACAC,Ipratropium bromide 500micrograms/2ml neb liq ud vials,42022,"597,169.05"
9,0301020I0AAAFAF,Ipratropium bromide 250micrograms/1ml neb liq ud vials,16225,"298,477.10"


For many of these it is likely the vast majority of the spend is on the UK licensed product - with only a small amount for specific patient cases - e.g. levothyroxine lactose free imported.
We may just need to exclude data for these products from any measure as we cannot reliably quantify the usage of imported vs non-imported from EPD.

We also need to consider mixed specials/imports products with UK licensed products

In [21]:
sql = '''
SELECT 
    amp.bnf_code,
    SUM(CASE WHEN (amp.avail_restrict = 6) THEN 1 ELSE 0 END) AS special_count,
    SUM(CASE WHEN (amp.avail_restrict = 4) THEN 1 ELSE 0 END) AS import_count,
    SUM(CASE WHEN (amp.avail_restrict = 6 OR amp.avail_restrict = 4) THEN 1 ELSE 0 END) AS special_import_count,
    SUM(CASE WHEN (amp.avail_restrict != 6 AND amp.avail_restrict != 9 AND amp.avail_restrict != 4) THEN 1 ELSE 0 END) AS non_special_non_import_count,
    STRING_AGG(DISTINCT CASE WHEN (amp.avail_restrict = 6 OR amp.avail_restrict = 4) THEN amp.descr ELSE NULL END, ', ') AS special_import_names,
    STRING_AGG(DISTINCT CASE WHEN (amp.avail_restrict != 6 AND amp.avail_restrict != 9 AND amp.avail_restrict != 4) THEN amp.descr ELSE NULL END, ', ') AS non_special_import_names,
    STRING_AGG(DISTINCT vmp.nm, ', ') AS vpm_names
FROM 
    `ebmdatalab.dmd.amp` AS amp
LEFT JOIN 
    `ebmdatalab.dmd.vmp` AS vmp
ON 
    amp.bnf_code = vmp.bnf_code
WHERE
    amp.bnf_code IS NOT NULL
    AND
    amp.bnf_code != '190203000BBAAA0' --  Exclude homeopathic products as total mix of specials and none specials but minimal impact on cost ~£2600 across England over 12 months
GROUP BY 
    amp.bnf_code
HAVING
    special_count > 0
    AND
    special_import_count >0
    AND
    non_special_non_import_count > 0
ORDER BY 
    special_import_count DESC;
'''
mixed_count_named_DF = bq.cached_read(sql, csv_path=os.path.join('../','data','mixed_count_named.csv'))

with pd.option_context('display.max_rows', 100):
    display(mixed_count_named_DF.head(100))

,bnf_code,special_count,import_count,special_import_count,non_special_non_import_count,special_import_names,non_special_import_names,vpm_names
0,0401010ADAACZCZ,1,0,1,7,Melatonin 1mg/ml oral solution alcohol free sugar free (Special Order),"Melatonin 1mg/ml oral solution sugar free (Colonis Pharma Ltd), Melatonin 1mg/ml oral solution sugar free (A A H Pharmaceuticals Ltd), Melatonin 1mg/ml oral solution sugar free (Alliance Healthcare (Distribution) Ltd), Melatonin 1mg/ml oral solution sugar free (Consilient Health Ltd), Melatonin 1mg/ml oral solution sugar free (Aspire Pharma Ltd), Melatonin 1mg/ml oral solution sugar free (Alissa Healthcare Research Ltd), Melatonin 1mg/ml oral solution sugar free (Thame Laboratories Ltd)",Melatonin 1mg/ml oral solution sugar free
1,0407010H0AAA5A5,1,0,1,4,Paracetamol 500mg/5ml oral suspension (Royal Preston Hospital mucilage formula) (Special Order),"Paracetamol 500mg/5ml oral suspension sugar free (Rosemont Pharmaceuticals Ltd), Paracetamol 500mg/5ml oral suspension sugar free (Alliance Healthcare (Distribution) Ltd), Paracetamol 500mg/5ml oral suspension sugar free (A A H Pharmaceuticals Ltd), Paracetamol 500mg/5ml oral suspension sugar free (Medihealth (Northern) Ltd)",Paracetamol 500mg/5ml oral suspension sugar free
2,0410030C0AAAAAA,1,0,1,5,Methadone 1mg/ml oral solution colour free (Special Order),"Methadone 1mg/ml oral solution (A A H Pharmaceuticals Ltd), Methadone 1mg/ml oral solution (Martindale Pharmaceuticals Ltd), Methadone 1mg/ml oral solution (Rosemont Pharmaceuticals Ltd), Methadone 1mg/ml oral solution (Alliance Healthcare (Distribution) Ltd), Methadone 1mg/ml oral solution (Thornton & Ross Ltd)",Methadone 1mg/ml oral solution
3,0410030C0AAAFAF,1,0,1,5,Methadone 1mg/ml oral solution colour free sugar free (Special Order),"Methadone 1mg/ml oral solution sugar free (A A H Pharmaceuticals Ltd), Methadone 1mg/ml oral solution sugar free (Martindale Pharmaceuticals Ltd), Methadone 1mg/ml oral solution sugar free (Rosemont Pharmaceuticals Ltd), Methadone 1mg/ml oral solution sugar free (Alliance Healthcare (Distribution) Ltd), Methadone 1mg/ml oral solution sugar free (Thornton & Ross Ltd)",Methadone 1mg/ml oral solution sugar free
4,0702020T0AAAFAF,1,0,1,1,"Nystatin 100,000unit pessaries (Special Order)","Nystatin 100,000unit pessaries (Sanofi)","Nystatin 100,000unit pessaries"
5,090501300AABIBI,1,0,1,1,Magnesium malate 250mg capsules (Special Order),Magnesium malate 250mg capsules (BioCare Ltd),Magnesium malate 250mg capsules
6,0905013F0AAABAB,1,0,1,1,Magnesium citrate 100mg capsules (Special Order),Magnesium citrate 100mg capsules (Vega Nutritionals Ltd),Magnesium citrate 100mg capsules
7,090504100AABBBB,1,0,1,1,Zinc citrate 50mg capsules (Special Order),Zinc citrate 50mg capsules (Vega Nutritionals Ltd),Zinc citrate 50mg capsules
8,0906012B0AAACAC,1,0,1,2,Betacarotene 15mg capsules (Special Order),"Betacarotene 15mg capsules (Lamberts Healthcare Ltd), Betacarotene 15mg capsules (Nature's Best Health Products Ltd)",Betacarotene 15mg capsules
9,0906022K0AAAPAP,1,0,1,1,Nicotinamide 250mg tablets (Special Order),Nicotinamide 250mg tablets (Lamberts Healthcare Ltd),Nicotinamide 250mg tablets


Only a small number of products. Add spend column to understand significance of the spending against that BNF code.

In [22]:
sql = '''
SELECT bnf_code, 
       bnf_name, 
       SUM(items) AS items, 
       SUM(actual_cost) AS cost
FROM `ebmdatalab.hscic.normalised_prescribing`
WHERE bnf_code IN (
    WITH aggregated_data AS (
        SELECT 
            amp.bnf_code,
            SUM(CASE WHEN (amp.avail_restrict = 6) THEN 1 ELSE 0 END) AS special_count,
            SUM(CASE WHEN (amp.avail_restrict = 4) THEN 1 ELSE 0 END) AS import_count,
            SUM(CASE WHEN (amp.avail_restrict = 6 OR amp.avail_restrict = 4) THEN 1 ELSE 0 END) AS special_import_count,
            SUM(CASE WHEN (amp.avail_restrict != 6 AND amp.avail_restrict != 9 AND amp.avail_restrict != 4) THEN 1 ELSE 0 END) AS non_special_non_import_count
        FROM 
            `ebmdatalab.dmd.amp` AS amp
        WHERE
            amp.bnf_code IS NOT NULL
            AND amp.bnf_code != '190203000BBAAA0'
        GROUP BY 
            amp.bnf_code
    )
    SELECT DISTINCT bnf_code
    FROM aggregated_data
    WHERE special_count > 0
    AND
    special_import_count >0
    AND
    non_special_non_import_count > 0
    
)
AND month >='2023-11-01'
AND month < '2024-11-01'
GROUP BY bnf_code, bnf_name
ORDER BY cost DESC
'''
mixed_specials_DF = bq.cached_read(sql, csv_path=os.path.join('../','data','mixed_specials.csv'))
with pd.option_context('display.max_rows', 100):
    with pd.option_context('display.float_format', '{:,.2f}'.format):
        display(mixed_specials_DF.head(100))

,bnf_code,bnf_name,items,cost
0,0401010ADAACZCZ,Melatonin 1mg/ml oral solution sugar free,123236,"13,883,526.23"
1,0410030C0AAAFAF,Methadone 1mg/ml oral solution sugar free,705950,"4,256,744.13"
2,0410030C0AAAAAA,Methadone 1mg/ml oral solution,214491,"1,312,512.73"
3,0906025P0AABFBF,Riboflavin 50mg capsules,4849,"706,847.98"
4,0407010H0AAA5A5,Paracetamol 500mg/5ml oral suspension sugar free,6863,"653,555.52"
5,0906022K0AAAPAP,Nicotinamide 250mg tablets,2891,"357,191.72"
6,0905013F0AAABAB,Magnesium citrate 100mg capsules,3559,"228,914.70"
7,0702020T0AAAFAF,"Nystatin 100,000unit pessaries",487,"179,385.75"
8,090504100AABBBB,Zinc citrate 50mg capsules,1037,"11,938.72"
9,0906012B0AAACAC,Betacarotene 15mg capsules,16,"6,659.56"


Again, for many of these it is likely the vast majority of the spend is on the UK licensed product

### Building final query
From the profiling above, for this measure we will look to include all BNF codes for special products or imported products except when they cannot be reliably differentiated from UK licensed products.
We want to include:
- all BNF codes where specials + import count > 0

Except:
- where non_special_non_import_count also >0

#### Excluded results

In [23]:
sql = '''
SELECT 
    amp.bnf_code,
    SUM(CASE WHEN (amp.avail_restrict = 6) THEN 1 ELSE 0 END) AS special_count,
    SUM(CASE WHEN (amp.avail_restrict = 4) THEN 1 ELSE 0 END) AS import_count,
    SUM(CASE WHEN (amp.avail_restrict = 6 OR amp.avail_restrict = 4) THEN 1 ELSE 0 END) AS special_import_count,
    SUM(CASE WHEN (amp.avail_restrict != 6 AND amp.avail_restrict != 9 AND amp.avail_restrict != 4) THEN 1 ELSE 0 END) AS non_special_non_import_count,
    STRING_AGG(DISTINCT CASE WHEN (amp.avail_restrict = 6 OR amp.avail_restrict = 4) THEN amp.descr ELSE NULL END, ', ') AS special_import_names,
    STRING_AGG(DISTINCT CASE WHEN (amp.avail_restrict != 6 AND amp.avail_restrict != 9 AND amp.avail_restrict != 4) THEN amp.descr ELSE NULL END, ', ') AS non_special_import_names,
    STRING_AGG(DISTINCT vmp.nm, ', ') AS vpm_names
FROM 
    `ebmdatalab.dmd.amp` AS amp
LEFT JOIN 
    `ebmdatalab.dmd.vmp` AS vmp
ON 
    amp.bnf_code = vmp.bnf_code
WHERE
    amp.bnf_code IS NOT NULL
GROUP BY 
    amp.bnf_code
HAVING
    special_import_count > 0
    AND
    non_special_non_import_count > 0
ORDER BY 
    special_import_count DESC;
'''
excluded_list_DF = bq.cached_read(sql, csv_path=os.path.join('../','data','excluded_list.csv'))

with pd.option_context('display.max_rows', 100):
    display(excluded_list_DF.head(100))

bnf_code  special_count  import_count  special_import_count  \
0   190203000BBAAA0          16238          5648                 21886   
1   0401010ADAABRBR              0             2                     2   
2   0102000J0AAAAAA              0             1                     1   
3   0203020P0AAANAN              0             1                     1   
4   0301020I0AAACAC              0             1                     1   
5   0301020I0AAAFAF              0             1                     1   
6   0401010ADAAAEAE              0             1                     1   
7   0401010ADAABSBS              0             1                     1   
8   0401010ADAACZCZ              1             0                     1   
9   0407010H0AAA5A5              1             0                     1   
10  0410030C0AAAAAA              1             0                     1   
11  0410030C0AAAFAF              1             0                     1   
12  0502050B0AAABAB              0             1                     1   
13  0502050B0AAACAC              0             1                     1   
14  0602010V0AABWBW              0             1                     1   
15  0602010V0AABXBX              0             1                     1   
16  0602010V0AABZBZ              0             1                     1   
17  0604011K0AAABAB              0             1                     1   
18  0605010P0AAAHAH              0             1                     1   
19  0702020T0AAAFAF              1             0                     1   
20  090501300AABIBI              1             0                     1   
21  0905013F0AAABAB              1             0                     1   
22  090504100AABBBB              1             0                     1   
23  0906012B0AAACAC              1             0                     1   
24  0906022K0AAAPAP              1             0                     1   
25  0906025P0AABFBF              1             0                     1   
26  0913445A0AAAGAG              1             0                     1   

    non_special_non_import_count  \
0                          32123   
1                              3   
2                              6   
3                              3   
4                              6   
5                              6   
6                              4   
7                              3   
8                              7   
9                              4   
10                             5   
11                             5   
12                             3   
13                             3   
14                            11   
15                            11   
16                            12   
17                             2   
18                             1   
19                             1   
20                             1   
21                             1   
22                             1   
23                             2   
24                             1   
25                             2   
26                             1   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

#### Included results

In [24]:
sql = '''
SELECT 
    amp.bnf_code,
    SUM(CASE WHEN (amp.avail_restrict = 6) THEN 1 ELSE 0 END) AS special_count,
    SUM(CASE WHEN (amp.avail_restrict = 4) THEN 1 ELSE 0 END) AS import_count,
    SUM(CASE WHEN (amp.avail_restrict = 6 OR amp.avail_restrict = 4) THEN 1 ELSE 0 END) AS special_import_count,
    SUM(CASE WHEN (amp.avail_restrict != 6 AND amp.avail_restrict != 9 AND amp.avail_restrict != 4) THEN 1 ELSE 0 END) AS non_special_non_import_count,
    STRING_AGG(DISTINCT CASE WHEN (amp.avail_restrict = 6 OR amp.avail_restrict = 4) THEN amp.descr ELSE NULL END, ', ') AS special_import_names,
    STRING_AGG(DISTINCT CASE WHEN (amp.avail_restrict != 6 AND amp.avail_restrict != 9 AND amp.avail_restrict != 4) THEN amp.descr ELSE NULL END, ', ') AS non_special_import_names,
    STRING_AGG(DISTINCT vmp.nm, ', ') AS vpm_names
FROM 
    `ebmdatalab.dmd.amp` AS amp
LEFT JOIN 
    `ebmdatalab.dmd.vmp` AS vmp
ON 
    amp.bnf_code = vmp.bnf_code
WHERE
    amp.bnf_code IS NOT NULL
GROUP BY 
    amp.bnf_code
HAVING
    special_import_count > 0
    AND
    non_special_non_import_count = 0
ORDER BY 
    special_import_count DESC;
'''
included_list_DF = bq.cached_read(sql, csv_path=os.path.join('../','data','included_list.csv'))
included_list_DF

,bnf_code,special_count,import_count,special_import_count,non_special_non_import_count,special_import_names,non_special_import_names,vpm_names
0,1305020C0AAFPFP,36,0,36,0,"Coal tar solution 6% / Salicylic acid 2% in Emulsifying ointment (Drug Tariff Special Order), Coal tar solution 6% / Salicylic acid 2% in Unguentum M cream (Drug Tariff Special Order), Coal tar solution 6% / Salicylic acid 2% in ointment (Drug Tariff Special Order), Coal tar solution 6% / Salicylic acid 2% in White soft paraffin (Drug Tariff Special Order), Coal tar solution 6% / Salicylic acid 2% in Yellow soft paraffin (Drug Tariff Special Order), Coal tar solution 6% / Salicylic acid 2% in Epaderm ointment (Drug Tariff Special Order)",NaN,"Coal tar solution 6% / Salicylic acid 2% in Emulsifying ointment, Coal tar solution 6% / Salicylic acid 2% in Generic Unguentum M cream, Coal tar solution 6% / Salicylic acid 2% in ointment, Coal tar solution 6% / Salicylic acid 2% in White soft paraffin, Coal tar solution 6% / Salicylic acid 2% in Yellow soft paraffin, Coal tar solution 6% / Salicylic acid 2% in Generic Epaderm ointment"
1,1305020C0AAAVAV,16,0,16,0,"Coal tar 5% in Yellow soft paraffin (Part VIIID Drug Tariff Special Order), Coal tar 5% in Emulsifying ointment (Part VIIID Drug Tariff Special Order), Coal tar 5% in White soft paraffin (Part VIIID Drug Tariff Special Order), Coal tar 5% in Hydrous ointment (Part VIIID Drug Tariff Special Order)",NaN,"Coal tar 5% in Emulsifying ointment, Coal tar 5% in White soft paraffin, Coal tar 5% in Hydrous ointment, Coal tar 5% in Yellow soft paraffin"
2,1305020C0AABSBS,16,0,16,0,"Coal tar 10% in Yellow soft paraffin (Part VIIID Drug Tariff Special Order), Coal tar 10% in Emulsifying ointment (Part VIIID Drug Tariff Special Order), Coal tar 10% in White soft paraffin (Part VIIID Drug Tariff Special Order), Coal tar 10% in Hydrous ointment (Part VIIID Drug Tariff Special Order)",NaN,"Coal tar 10% in Yellow soft paraffin, Coal tar 10% in Emulsifying ointment, Coal tar 10% in White soft paraffin, Coal tar 10% in Hydrous ointment"
3,1305020S0AACZCZ,16,0,16,0,"Salicylic acid 5% in Emulsifying ointment (Drug Tariff Special Order), Salicylic acid 5% in White soft paraffin (Drug Tariff Special Order), Salicylic acid 5% in Yellow soft paraffin (Drug Tariff Special Order), Salicylic acid 5% in Hydrous ointment (Drug Tariff Special Order)",NaN,"Salicylic acid 5% in Emulsifying ointment, Salicylic acid 5% in White soft paraffin, Salicylic acid 5% in Yellow soft paraffin, Salicylic acid 5% in Hydrous ointment"
4,1307000M0AABVBV,16,0,16,0,"Salicylic acid 10% in Emulsifying ointment (Drug Tariff Special Order), Salicylic acid 10% in Hydrous ointment (Drug Tariff Special Order), Salicylic acid 10% in White soft paraffin (Drug Tariff Special Order), Salicylic acid 10% in Yellow soft paraffin (Drug Tariff Special Order)",NaN,"Salicylic acid 10% in Emulsifying ointment, Salicylic acid 10% in Hydrous ointment, Salicylic acid 10% in White soft paraffin, Salicylic acid 10% in Yellow soft paraffin"
...,...,...,...,...,...,...,...,...
4706,190605000AACECE,1,0,1,0,Cade oil 12.5% in Emulsifying ointment (Special Order),NaN,Cade oil 12.5% in Emulsifying ointment
4707,190607000AAAGAG,1,0,1,0,Ginger syrup (Special Order),NaN,Ginger syrup
4708,190800000AAAPAP,1,0,1,0,Eosin 1% solution (Special Order),NaN,Eosin 1% solution
4709,190800000AABTBT,1,0,1,0,Eosin 2% solution (Drug Tariff Special Order),NaN,Eosin 2% solution


#### Included distinct BNF codes 

In [25]:
sql = '''
WITH aggregated_data AS (
    SELECT 
        amp.bnf_code,
        SUM(CASE WHEN (amp.avail_restrict = 6 OR amp.avail_restrict = 4) THEN 1 ELSE 0 END) AS special_import_count,
        SUM(CASE WHEN (amp.avail_restrict != 6 AND amp.avail_restrict != 9 AND amp.avail_restrict != 4) THEN 1 ELSE 0 END) AS non_special_non_import_count
    FROM 
        `ebmdatalab.dmd.amp` AS amp
    WHERE
        amp.bnf_code IS NOT NULL
    GROUP BY 
        amp.bnf_code
    )
SELECT DISTINCT bnf_code
FROM aggregated_data
WHERE special_import_count > 0
AND
non_special_non_import_count = 0;
'''
included_bnf_code_list_DF = bq.cached_read(sql, csv_path=os.path.join('../','data','included_bnf_code_list.csv'))
included_bnf_code_list_DF

,bnf_code
0,0101010I0BCAABI
1,0102000D0BHABAH
2,0102000R0BBAAAB
3,0103030S0BCAAAA
4,0103030S0BDAAAA
...,...
4706,190605000AACECE
4707,190607000AAAGAG
4708,190800000AAAPAP
4709,190800000AABTBT


In [26]:
sql = '''
WITH aggregated_data AS (
    SELECT 
        amp.bnf_code,
        SUM(CASE WHEN (amp.avail_restrict = 6 OR amp.avail_restrict = 4) THEN 1 ELSE 0 END) AS special_import_count,
        SUM(CASE WHEN (amp.avail_restrict != 6 AND amp.avail_restrict != 9 AND amp.avail_restrict != 4) THEN 1 ELSE 0 END) AS non_special_non_import_count
    FROM 
        `ebmdatalab.dmd.amp` AS amp
    WHERE
        amp.bnf_code IS NOT NULL
    GROUP BY 
        amp.bnf_code
    )
SELECT DISTINCT bnf_code
FROM aggregated_data
WHERE special_import_count > 0
AND
non_special_non_import_count = 0;
'''
included_bnf_code_list_DF = bq.cached_read(sql, csv_path=os.path.join('../','data','included_bnf_code_list.csv'))
included_bnf_code_list_DF

,bnf_code
0,0101010I0BCAABI
1,0102000D0BHABAH
2,0102000R0BBAAAB
3,0103030S0BCAAAA
4,0103030S0BDAAAA
...,...
4706,190605000AACECE
4707,190607000AAAGAG
4708,190800000AAAPAP
4709,190800000AABTBT
